In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import re
from splinter.exceptions import ElementDoesNotExist

from config import password
from sqlalchemy import create_engine
import platform

csv_path = "Resources/Amzon_All_Books.csv"


# Detecting OS

In [2]:
system = platform.system()
print(system)
if system == 'Darwin':
    cd = 'chromedriver'
elif system == 'Windows':
    cd = 'chromedrive.exe'


Darwin


In [3]:
executable_path = {'executable_path': cd}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
url = 'https://www.amazon.com/s?i=stripbooks&s=relevanceexprank&Adv-Srch-Books-Submit.x=35&Adv-Srch-Books-Submit.y=8&unfiltered=1&ref=sr_adv_b'
browser.visit(url)


In [5]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [6]:
results = []

In [8]:
def scrapeWebPage():
    try:
       
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        parents = soup.select("div[data-asin]")   
        for parent in parents:
            my_dict = {}
            my_dict["title"] = parent.select(".a-color-base.a-text-normal")[0].text
            rating = parent.select(".a-icon-alt")
            
            if len(rating) == 0:
                 my_dict["rating"] = 'N/A'
            else: 
                my_dict["rating"] = rating[0].text.split(' ')[0]
            author = parent.select(".a-color-secondary .a-size-base+ .a-size-base")
            #.a-color-secondary .a-size-base.a-link-normal
            #.a-color-secondary .a-size-base+ .a-size-base
            if(len(author)== 0):
                my_dict["author"] = 'N/A'
            else:
                my_dict["author"] =  author[0].text.strip()

            review_count = parent.select(".a-size-small .a-link-normal .a-size-base")
            if(len(review_count)== 0):
                my_dict["review count"] = 0
            else:
                my_dict["review count"] =  int(review_count[0].text.replace(',',''))
            results.append(my_dict)
            
            notLastPage = True
        if(browser.find_by_css(".a-last a")):
            browser.find_by_css(".a-last a").click()
        else:
            notLastPage = False
        return notLastPage
    except Exception as e:
        print(f"Exception !!!  {e}")
        
notLastPage = True
forcedExit = 0
while notLastPage:
    notLastPage = scrapeWebPage()
#     forcedExit += 1
#     if forcedExit == 2:
#         notLastPage = False

In [9]:
print(len(results))
print(type(results))


1200
<class 'list'>


In [11]:
amazon_all_books_db = pd.DataFrame(results)
amazon_all_books_db.columns = ['authors', 'avg_rating', 'ratings_count', 'title']
 
# Set index to book_id
amazon_all_books_db.index.name = 'book_id'
amazon_all_books_db.head(10)

,authors,avg_rating,ratings_count,title
book_id,,,,
0,Delia Owens,4.8,19380,Where the Crawdads Sing
1,IglooBooks,N/A,0,Disney: Storybook Collection Advent Calendar
2,Daniel Silva,4.6,631,The New Girl: A Novel (Gabriel Allon Book 19)
3,Tara Westover,4.7,8741,Educated: A Memoir
4,David Baldacci,4.1,318,One Good Deed
5,Elin Hilderbrand,4.3,590,Summer of '69
6,Mariana Zapata,4.7,293,The Best Thing
7,Dav Pilkey,4.8,59,Dog Man: For Whom the Ball Rolls: From the Cre...
8,Nora Roberts,4.5,569,Under Currents: A Novel


# Writing to csv

In [12]:
amazon_all_books_db.to_csv(csv_path,index=False,header=True)

# Data Base 

In [13]:
# Create Postgres engine
engine = create_engine('postgresql://postgres:'+ password + '@localhost:5432/books_db')

In [14]:
# Read tables
engine.table_names()

['goodreads', 'amazon']

In [15]:
# Insert data into postgres table
amazon_all_books_db.to_sql(name='amazon', con=engine, if_exists='replace', index=True)

In [16]:
# Confirm data stored in the database
pd.read_sql_query('select * from amazon', con=engine).head()

,book_id,authors,avg_rating,ratings_count,title
0,0,Delia Owens,4.8,19380,Where the Crawdads Sing
1,1,IglooBooks,N/A,0,Disney: Storybook Collection Advent Calendar
2,2,Daniel Silva,4.6,631,The New Girl: A Novel (Gabriel Allon Book 19)
3,3,Tara Westover,4.7,8741,Educated: A Memoir
4,4,David Baldacci,4.1,318,One Good Deed
